### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Spanish
Lang Id = 4
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,eso de lo que es la la el el,5747,5753,eso de lo que es la la el el,UVeOYNyXd0k,https://www.youtube.com/watch?v=UVeOYNyXd0k&t=...
1,si lo que se está es una un,23,25,clama y al si lo que se está es una un,rP0PowPzbJE,https://www.youtube.com/watch?v=rP0PowPzbJE&t=23s
2,es que por qué no para en,2158,2160,es que por qué no para en casa,l1fO8CJN7Bw,https://www.youtube.com/watch?v=l1fO8CJN7Bw&t=...
3,de por sí sí es un que,979,982,de por sí sí es un que viene,kqdoxv8Ft7Q,https://www.youtube.com/watch?v=kqdoxv8Ft7Q&t=...
4,la una que es para los,98,103,desde la una que es para los,2DvDa1DL9iM,https://www.youtube.com/watch?v=2DvDa1DL9iM&t=98s
...,...,...,...,...,...,...
195,está en el,343,347,está en el top,iRZvnTlrj64,https://www.youtube.com/watch?v=iRZvnTlrj64&t=...
196,pero no lo,723,727,pero no lo son,2vm9WQyv9n8,https://www.youtube.com/watch?v=2vm9WQyv9n8&t=...
197,pero a los,308,311,pero a los,NIBRNVAqiFI,https://www.youtube.com/watch?v=NIBRNVAqiFI&t=...
198,pero el de,225,230,pero el de,UJRYvK6Asug,https://www.youtube.com/watch?v=UJRYvK6Asug&t=...


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,porque lo tienes todo pero a la vez es como qu...,685,692,porque lo tienes todo pero a la vez es como qu...,ZyghXM2t1CM,https://www.youtube.com/watch?v=ZyghXM2t1CM&t=...
1,y eso es bueno ahora tenemos un mejor dinero p...,151,159,y eso es bueno ahora tenemos un mejor dinero p...,jX_pbV6ea3c,https://www.youtube.com/watch?v=jX_pbV6ea3c&t=...
2,aquí yo no lo voy a hacer porque lo voy a hace...,1795,1801,vamos a tejer aquí yo no lo voy a hacer porque...,ujmlPjCimbM,https://www.youtube.com/watch?v=ujmlPjCimbM&t=...
3,o sea que este es el trabajo de la noche pero ...,870,875,o sea que este es el trabajo de la noche pero ...,aL1sepsZoT0,https://www.youtube.com/watch?v=aL1sepsZoT0&t=...
4,mucho más o menos así más pero que están muy b...,158,161,molan mucho más o menos así más pero que están...,MeAGI1vCdN4,https://www.youtube.com/watch?v=MeAGI1vCdN4&t=...
...,...,...,...,...,...,...
195,la vida es un tiempo pero el tiempo se me va,1241,1245,la vida es un tiempo pero el tiempo se me va,vrioD2Abq00,https://www.youtube.com/watch?v=vrioD2Abq00&t=...
196,para hablar de eso pero sí hay mucho trabajo,252,257,para hablar de eso pero sí hay mucho trabajo duro,70G3qNCLEq4,https://www.youtube.com/watch?v=70G3qNCLEq4&t=...
197,gracias me hace menos pero aquí tenemos está,619,625,gracias me hace menos pero aquí tenemos está,i9Vr601ELbE,https://www.youtube.com/watch?v=i9Vr601ELbE&t=...
198,gracias por estar ahí una vez más y hasta la,793,836,muchas gracias por estar ahí una vez más y has...,l3xxCR3LGaI,https://www.youtube.com/watch?v=l3xxCR3LGaI&t=...


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,también dice que se siente solo porque no tien...,250,256,también dice que se siente solo porque no tien...,QGlH1rRdROo,https://www.youtube.com/watch?v=QGlH1rRdROo&t=...
1,así que creo que todavía hay algunas preguntas...,1122,1129,así que creo que todavía hay algunas preguntas...,O0rkwLyf3BA,https://www.youtube.com/watch?v=O0rkwLyf3BA&t=...
2,el problema es que tienen la mejor arma del mu...,744,751,el problema es que tienen la mejor arma del mu...,fl70SaNinZs,https://www.youtube.com/watch?v=fl70SaNinZs&t=...
3,si tienes razón pero en realidad te lo dice po...,436,443,si tienes razón pero en realidad te lo dice po...,NWsd2DM3akQ,https://www.youtube.com/watch?v=NWsd2DM3akQ&t=...
4,y eso es bueno ahora tenemos un mejor dinero p...,151,160,y eso es bueno ahora tenemos un mejor dinero p...,jX_pbV6ea3c,https://www.youtube.com/watch?v=jX_pbV6ea3c&t=...
...,...,...,...,...,...,...
195,por aquí o sea este día no puede terminar peor...,431,437,por aquí o sea este día no puede terminar peor...,ebT4f30kry4,https://www.youtube.com/watch?v=ebT4f30kry4&t=...
196,sí pero nosotros otro y yo nos hemos vuelto de...,135,141,sí pero nosotros otro y yo nos hemos vuelto de...,v9hx45VZn_U,https://www.youtube.com/watch?v=v9hx45VZn_U&t=...
197,pero al final todos la hemos visto eso también...,1074,1080,pero al final todos la hemos visto eso también...,ZFndiQe4lMk,https://www.youtube.com/watch?v=ZFndiQe4lMk&t=...
198,está bien si tienes una idea pero no estás seg...,374,380,está bien si tienes una idea pero no estás seg...,LTi6cyrVpwE,https://www.youtube.com/watch?v=LTi6cyrVpwE&t=...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,y además cómo sabría el tipo cuándo ha cumplid...,207,219,y además cómo sabría el tipo cuándo ha cumplid...,wFEtvZmaNwQ,https://www.youtube.com/watch?v=wFEtvZmaNwQ&t=...
1,por lo tanto desea comenzar a hacer cosas con ...,490,501,por lo tanto desea comenzar a hacer cosas con ...,fcxTPwMyaNs,https://www.youtube.com/watch?v=fcxTPwMyaNs&t=...
2,y como la tiene en su poder se la vende a max ...,196,206,y como la tiene en su poder se la vende a max ...,zTxdXfa8zlA,https://www.youtube.com/watch?v=zTxdXfa8zlA&t=...
3,desde luego no es mala idea porque él no está ...,425,433,desde luego no es mala idea porque él no está ...,Jw5JFE_7icI,https://www.youtube.com/watch?v=Jw5JFE_7icI&t=...
4,la casa es donde está el corazón espero que us...,479,490,la casa es donde está el corazón espero que us...,NzYHZFCpNag,https://www.youtube.com/watch?v=NzYHZFCpNag&t=...
...,...,...,...,...,...,...
195,bueno estamos hoy aquí con todos vosotros para...,33,38,bueno estamos hoy aquí con todos vosotros para...,_LZkYgkOcGo,https://www.youtube.com/watch?v=_LZkYgkOcGo&t=33s
196,complicado chicos no sé podré mía esto es enor...,963,970,complicado chicos no sé podré mía esto es enor...,zz98Q9ce5Wc,https://www.youtube.com/watch?v=zz98Q9ce5Wc&t=...
197,dar un poquito como para meter un poquito haci...,129,135,dar un poquito como para meter un poquito haci...,VHNmlwcmTZY,https://www.youtube.com/watch?v=VHNmlwcmTZY&t=...
198,cuando se todos los días puede ver una situaci...,384,389,cuando se todos los días puede ver una situaci...,OlcY8gl8jJc,https://www.youtube.com/watch?v=OlcY8gl8jJc&t=...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8,https://www.youtube.com/watch?v=ZMOu81BhyH8&t=46s
1,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E,https://www.youtube.com/watch?v=xqr2skItS3E&t=...
2,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA,https://www.youtube.com/watch?v=zTxdXfa8zlA&t=...
3,qué pasa el chaval jamás hasta ese momento ha ...,84,93,qué pasa el chaval jamás hasta ese momento ha ...,3dc1Dwjlwys,https://www.youtube.com/watch?v=3dc1Dwjlwys&t=84s
4,desde luego no es mala idea porque él no está ...,425,433,desde luego no es mala idea porque él no está ...,Jw5JFE_7icI,https://www.youtube.com/watch?v=Jw5JFE_7icI&t=...
...,...,...,...,...,...,...
195,para hacer las cosas como las tenemos hasta es...,3722,3733,para hacer las cosas como las tenemos hasta es...,BbqBQdL0Sow,https://www.youtube.com/watch?v=BbqBQdL0Sow&t=...
196,para saludar como quien dice pues hola pues va...,1591,1598,para saludar como quien dice pues hola pues va...,9NzW81HnRSU,https://www.youtube.com/watch?v=9NzW81HnRSU&t=...
197,tenemos mucho más mérito cuando hacemos un gol...,176,183,tenemos mucho más mérito cuando hacemos un gol...,h0W5xKRj3lw,https://www.youtube.com/watch?v=h0W5xKRj3lw&t=...
198,dorado no sé bien qué color es pero pues tambi...,181,188,dorado no sé bien qué color es pero pues tambi...,HxmBhn57Gyc,https://www.youtube.com/watch?v=HxmBhn57Gyc&t=...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,pero es que si ese era tu plan para qué dejas ...,398,410,pero es que si ese era tu plan para qué dejas ...,Ur8vnsSGa7U,https://www.youtube.com/watch?v=Ur8vnsSGa7U&t=...
1,genial problema resuelto excepto que aquí es d...,254,264,genial problema resuelto excepto que aquí es d...,cTdqYOaZNtQ,https://www.youtube.com/watch?v=cTdqYOaZNtQ&t=...
2,todos estos cortes se pueden hacer usando una ...,70,81,todos estos cortes se pueden hacer usando una ...,ZCGlkmnwFaE,https://www.youtube.com/watch?v=ZCGlkmnwFaE&t=70s
3,incluso después de la muerte el cuerpo humano ...,218,232,incluso después de la muerte el cuerpo humano ...,oLuBvogL88U,https://www.youtube.com/watch?v=oLuBvogL88U&t=...
4,pero todavía podría usarla si cree que su pilo...,296,307,pero todavía podría usarla si cree que su pilo...,Xq64fmkeETo,https://www.youtube.com/watch?v=Xq64fmkeETo&t=...
...,...,...,...,...,...,...
195,gordito hemos estado comiendo bastante porque ...,259,270,gordito hemos estado comiendo bastante porque ...,hYOAqPWqa1g,https://www.youtube.com/watch?v=hYOAqPWqa1g&t=...
196,visitando berlín por primera vez este es un bu...,103,109,visitando berlín por primera vez este es un bu...,hYOAqPWqa1g,https://www.youtube.com/watch?v=hYOAqPWqa1g&t=...
197,lo usamos como dos veces solamente bueno enton...,526,533,lo usamos como dos veces solamente bueno enton...,UEfQ7vFGXsM,https://www.youtube.com/watch?v=UEfQ7vFGXsM&t=...
198,los muchachos te saquen mucha más información ...,392,400,los muchachos te saquen mucha más información ...,ZgqzZQa6gQM,https://www.youtube.com/watch?v=ZgqzZQa6gQM&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,o por qué no fabricar una bomba propia si tien...,438,452,o por qué no fabricar una bomba propia si tien...,haz50gsL2G4,https://www.youtube.com/watch?v=haz50gsL2G4&t=...
1,pero es que si ese era tu plan para qué dejas ...,398,410,pero es que si ese era tu plan para qué dejas ...,Ur8vnsSGa7U,https://www.youtube.com/watch?v=Ur8vnsSGa7U&t=...
2,no hay ninguna seguridad más porque conseguir ...,385,396,no hay ninguna seguridad más porque conseguir ...,haz50gsL2G4,https://www.youtube.com/watch?v=haz50gsL2G4&t=...
3,dependiendo del tipo de clip seleccionado en l...,548,561,dependiendo del tipo de clip seleccionado en l...,danPB1e3MhA,https://www.youtube.com/watch?v=danPB1e3MhA&t=...
4,en la parte incorrecta del motor lo que causa ...,449,462,encenderse en la parte incorrecta del motor lo...,30pBgk4cZU8,https://www.youtube.com/watch?v=30pBgk4cZU8&t=...
...,...,...,...,...,...,...
195,pregunta si tienes alguna duda siempre digo pu...,264,271,pregunta si tienes alguna duda siempre digo pu...,pJcF5lULCGA,https://www.youtube.com/watch?v=pJcF5lULCGA&t=...
196,quiero ganar una partida chicos o sea es mi su...,487,493,quiero ganar una partida chicos o sea es mi su...,dGkvPrFJ_B0,https://www.youtube.com/watch?v=dGkvPrFJ_B0&t=...
197,importante puede ser muy importante pero no ti...,619,626,importante puede ser muy importante pero no ti...,GWU6dmcjHnU,https://www.youtube.com/watch?v=GWU6dmcjHnU&t=...
198,nieve con carga distinta realmente hay una peq...,322,327,nieve con carga distinta realmente hay una peq...,x5RImUQK0w0,https://www.youtube.com/watch?v=x5RImUQK0w0&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1675330545
  nanos: 694309000
}